In [ ]:
!nvidia-smi

Tue Jan 23 04:08:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00


First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

lovenpiece = '/content/drive/MyDrive/MODU/nikluge-au-2022-train.json'
lovenpiece_test = '/content/drive/MyDrive/MODU/nikluge-au-2022-test.json'
train_dataset = load_dataset("json", data_files=lovenpiece)
test_dataset = load_dataset("json", data_files=lovenpiece_test)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
for i in train_dataset['train']:
  print(i)
  break

{'id': 'nikluge-au-2022-train-000001', 'input': '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..', 'output': 1}


In [ ]:

# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
# data = train_dataset.map(
#     lambda x: {'text': f"### 문장: {x['input']}\n\n### 유형: {'혐오' if (x['output']==1) else '비혐오'} \n\n### 수정된 문장: {x['right']}<|endoftext|>" }
# )
data = train_dataset.map(
    lambda x: {'text': f"### 문장: {x['input']}\n\n### 유형: {'혐오' if (x['output']==1) else '비혐오'}<|endoftext|>" }
)

Map:   0%|          | 0/16580 [00:00<?, ? examples/s]

In [ ]:
data['train']['text'][0]

'### 문장: 보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..\n\n### 유형: 혐오<|endoftext|>'

In [ ]:
from datasets import load_dataset

dataset_sft = load_dataset("kyujinpy/OpenOrca-KO")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21632 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
import torch
model_id = "beomi/OPEN-SOLAR-KO-10.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/87.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/16580 [00:00<?, ? examples/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
import json
from typing import List

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    similarity = intersection_size / union_size if union_size != 0 else 0
    return similarity

def filter_similar_data(json_data, threshold=0.4):
    filtered_data = []

    for entry in json_data:

        output_set = set(entry['output'])

        is_similar = False
        for existing_entry in filtered_data:

            existing_output_set = set(existing_entry['output'])


            output_similarity = jaccard_similarity(output_set, existing_output_set)

            # 모든 input과 output의 유사도가 threshold 이하인 경우에만 추가
            if output_similarity >= threshold:
                is_similar = True
                break

        if not is_similar:
            filtered_data.append(entry)

    return filtered_data


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
filtered_json_data = filter_similar_data(dataset_sft['train'], threshold=0.4)

In [ ]:
len(filtered_json_data)

11445

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'down_proj', 'gate_proj', 'k_proj', 'v_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 31457280 || all params: 5648027648 || trainable%: 0.5569604463805911


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
tokenizer.encode("### 문장: {x['input']}\n\n### 유형: 혐오")

[1,
 774,
 32037,
 29747,
 28747,
 371,
 28744,
 1410,
 2537,
 1421,
 28752,
 13,
 13,
 27332,
 44144,
 28747,
 35867]

In [ ]:
tokenizer.decode(28747)

':'

In [ ]:
# for i in range(10000):
#   if tokenizer.decode([i]) == '<혐오>':
#     print(i)
#     break

In [ ]:
tokenizer.decode([6, 6, 6, 5716, 29, 16922, 2])

"'''iber> defeat#"

In [ ]:
tokenNum_emo = 44144
tokenNum_colon= 28747

In [ ]:
import transformers
from transformers import Trainer
import numpy as np
class maskTrainer(Trainer):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

  def compute_loss(self, model, inputs, return_outputs=False):
    # print(inputs['labels'][1])
    for x in range(len(inputs['labels'])):
      maskindex = (inputs['labels'][x]==tokenNum_emo).nonzero()[:, 0]
      temp = 0
      for i, index in enumerate(maskindex):
        if (inputs['labels'][x][index+1] != tokenNum_colon):
          maskindex = np.delete(maskindex.cpu(), i-temp)
          temp += 1
      #실제 감정을 나타내는 토큰을 제외한 모든 토큰을 마스킹 처리합니다.
      inputs['labels'][x][:maskindex[0]+2] = -100
      # print(inputs['labels'][x])

    outputs = model(**inputs)
    loss = outputs['loss']

    return (loss,outputs) if return_outputs else loss

In [ ]:


# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = maskTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
       # warmup_steps=00,
        #max_steps=3000, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        fp16=True,
        output_dir="outputs",
        logging_steps=30,
        num_train_epochs = 2,
        learning_rate=1e-4,


        lr_scheduler_type= "cosine",
        #optim="paged_adamw_8bit"

    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=data["train"],
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=64,
#         gradient_accumulation_steps=1,
#         # warmup_steps=200,
#         #max_steps=3000, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
#         fp16=True,
#         output_dir="outputs",
#         logging_steps=30,
#         num_train_epochs = 3,
#         learning_rate=1.4e-3,


#         lr_scheduler_type= "cosine",
#         #optim="paged_adamw_8bit"

#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
30,0.297700
60,0.102000
90,0.107000
120,0.068600
150,0.072200
180,0.079800
210,0.090700
240,0.065100
270,0.069300
300,0.067100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
model.save_pretrained('drive/MyDrive/MODU/outputs2')

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
def gen(x):
    prompt = f"### 문장: {x}\n\n### 유형:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2,
        early_stopping=True,
        do_sample=True,
        eos_token_id=46100,
    )
    return 1 if (tokenizer.decode(gened[0]).replace(prompt+ " ", "")=="<s> 혐오<|endoftext|>") else 0

In [ ]:
def gen2(x):
    prompt = f"### 문장: {x}\n\n### 유형:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2,
        early_stopping=True,
        do_sample=True,
        eos_token_id=50256,
    )
    return tokenizer.decode(gened[0]).replace(prompt+ " ", "")

In [ ]:
answerlist = []

for i in test_dataset['train']:

  answerlist.append(gen(i['input']))

In [ ]:
# "극좌는 이 비겁자층을 제대로 요리할 줄 안다..."
# "내가 진짜 올 해 안에 차 산다!"
# "선거 때마다 불장난 하는 못된 버릇 대대손손 배워가지고 그러고 까불어대면, 너 나중에 뒷덜미에 혹난다???"

In [ ]:
import json
dataset_dict = test_dataset['train'].to_dict()
new_dataset  = []
new_line = {}
for i, value in enumerate(answerlist):
    new_line['id'] = dataset_dict['id'][i]
    new_line['input'] = dataset_dict['input'][i]
    new_line['output'] = answerlist[i]
    a =new_line.copy()
    new_dataset.append(a)

In [ ]:
with open("submissionssssss.json", "w") as file:
    for item in new_dataset:
      json.dump(item, file, ensure_ascii=False)
      file.write('\n')

In [ ]:
gen("악마의 다른 얼굴입니다.")

1